# Varying delays - data analysis

Analysis of the spectral data obtained for varying delays for the aluminium block (Miguel Ferreira).
Conclusions: there is a self-reversal present, however better predictions will require more lines: additional spectrometer range (possible but we should be aware that a new temperature is computed for each spectrometer); an optical system that does not block below 300nm;

In [1]:
%matplotlib widget
from core.experiment import *

## Load the data from folder

In [2]:
root_path = "D:\\Data_LIBS\\Miguel\\Exp delays\\2021-03-03\\Bloco de Aluminio\\"

######################################################################
folders = [f for f in os.listdir(root_path)]
folder = folders[0]
sample_folder = root_path + folder + "\\"
delay_experiment = experiment(sample_folder,ignore='0000')

spectrums = []
wavelengths =[]
shot_n = []

for i in range(0, len(delay_experiment.list_of_signals)):
    
    current_signal = delay_experiment.list_of_signals[i]
    spectrums.append(current_signal.spectrum)
    wavelengths.append(current_signal.wavelengths)
    shot_n.append(current_signal.shot_number)
    
spectrums = np.array(spectrums)[:,0,:]
wavelengths = np.array(wavelengths)[:,0,:]
shot_n= np.array(shot_n)

# Simple Plot

In [3]:
fig=subplots()
plot(wavelengths[100],spectrums[100])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 3d Plot
A routine for representing the variation against the delay time;

In [4]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors


minim = 304
maxim = 313

samples_bin=spectrums

#construct the delays array given the information
times = np.linspace(2970,3007,len(shot_n))
wave = wavelengths[0]

fig = plt.figure(figsize=[10,10])
ax = fig.gca(projection='3d')


xs = wave[find_wavelength_index(minim, wave):find_wavelength_index(maxim, wave)]

samples_3d_plot=np.array(samples_bin)
verts = []
zs = np.array(times)
xs_1=[]
ys_1=[]
zs_1=[]

for i in range(0,len(zs)):
    ys = samples_3d_plot[i][find_wavelength_index(minim, wave):find_wavelength_index(maxim, wave)]
    ys[0], ys[-1] = 0, 0
    verts.append(list(zip(xs, ys)))
    
    xs_1.append(xs)
    ys_1.append(np.ones(xs.shape)*zs[i])
    zs_1.append(samples_3d_plot[i][find_wavelength_index(minim, wave):find_wavelength_index(maxim, wave)])
    

poly = PolyCollection(verts)
poly.set_alpha(0.7)
#ax.add_collection3d(poly, zs=zs, zdir='y')

for i in range(0,len(zs)):
    
    if i%10==0:
        ax.plot(xs_1[i],ys_1[i],zs_1[i],'k-',alpha=1.)
    elif i%2==0:
        ax.plot(xs_1[i],ys_1[i],zs_1[i],'k-',alpha=0.2)


ax.set_xlabel('$\lambda$ (nm)',fontsize=16)
ax.set_xlim3d(minim, maxim)
ax.set_ylabel('gate delay $t_{i}$ (ns)',fontsize=16)
ax.set_ylim3d(times[0], times[-1])#1.)
ax.set_zlabel('Counts',fontsize=16)
ax.set_zlim3d(0, samples_3d_plot[:,find_wavelength_index(minim, wave):find_wavelength_index(maxim, wave)].max())#25000)
#ax.set_zticks([0,12500,25000])


ax.get_proj = lambda: np.dot(Axes3D.get_proj(ax), np.diag([1,1,0.5, 1]))

#ax.view_init(40, -40)
#ax.grid(False)
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
#ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Computing the variation of a given line against 

# Computing the temperature

In [37]:
#Choose the element for computing the relevant lines
al = element("Al")

#reference temperature
Tp = 1.*T_ref
density = n_e_ref

##signal
sg = delay_experiment.list_of_signals[-35]

#define the upper and lower limits to find the most relevant nlines
#let us define to the
n_spectrometer = 0
ll = 300
ul = sg.wavelengths[n_spectrometer][-1]

#ion state 1
nlines = 3
l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#compute the plasma temperature
temperature, temp_95, r2, y_s, x_s = sg.saha_boltzmann_temperature_v2([lines],ion_energies,electron_density= density, 
                                                                      ratio_of_maximum = 0.5, radius = 0.1, Plot = True, Plotlines = True, use_max_intensity = False)


 Warning, asked for 3 only got 3


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Compare with a synthetic spectrum

In [34]:
sample_elements = [['Al',1]]
sample = digital_twin(sample_elements)

print(sample)

temperature =1*T_ref
sg = delay_experiment.list_of_signals[-35]
sg.remove_baseline()

for n_spectrometer in range(0,len(sg.wavelengths)):
    ll = sg.wavelengths[n_spectrometer][0]
    ul = sg.wavelengths[n_spectrometer][-1]

    #ion state 1
    nlines = 3
    l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, 
                                                        n_lines = nlines,lower_limit = ll, upper_limit = ul)

    #line to normalize the intensity for direct
    line_norm1=lines[0]
    print("# Spectrometer 1 normalized to " + str(line_norm1))

    sg.compare_to_digital_sample(sample, spectrometer=n_spectrometer, 
                                 electron_temperature = temperature,electron_density=density,
                                 d_lambda=0.02, use_wavelengths=True,line_normalize=line_norm1,
                                 resolution = 1000, Plotline = True)

Digital sample with 
 [ Al ratio 1 ] 
# Spectrometer 1 normalized to Line of Wavelength 309.27083999999996



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Computing density
Using the H_alpha line from the Balmer series.

In [ ]:
figure(13)
for i in range(0,len(list_of_signals)):
    sg=list_of_signals[i].mean_signal
    plot(sg.wavelengths[0], sg.spectrum[0], '-', label = str(integration_time[i]))

In [ ]:
list_of_signals[0].mean_signal.spectrum[0]

# Computing the plasma temperature
Using the Saha-Boltzmann method and choosing the lines

Using Na

In [ ]:
#Choose the element for computing the relevant lines
al = element('Na')

Tp = 1.*T_ref

#define the upper and lower limits to find the most relevant nlines
#let us define to the
n_spectrometer = 5
ll = sg.wavelengths[n_spectrometer][0]
ul = sg.wavelengths[n_spectrometer][-1]

#ion state 1
nlines = 5
l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#ion state 2
nlines = 2
l_num2, lines2 = al.get_most_relevant_lines_ion_state(ion_state = 2, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#ion state 3
nlines = 2
l_num3, lines3 = al.get_most_relevant_lines_ion_state(ion_state = 3, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)
ion_energies = al.ion_energies

#compute the plasma temperature
temperature, temp_95, r2, y_s, x_s = sg.saha_boltzmann_temperature_v2([lines,lines2],ion_energies,electron_density= density, 
                                                                      ratio_of_maximum = 0.5, radius = 0.1, Plot = True, Plotlines = True, use_max_intensity = False)


Using Cl

In [ ]:
#Choose the element for computing the relevant lines
al = element('Cl')

Tp = 1.*T_ref

#define the upper and lower limits to find the most relevant nlines
#let us define to the
n_spectrometer = 4
ll = sg.wavelengths[n_spectrometer][0]
ul = sg.wavelengths[n_spectrometer][-1]

#ion state 1
nlines = 5
l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#ion state 2
nlines = 2
l_num2, lines2 = al.get_most_relevant_lines_ion_state(ion_state = 2, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

#ion state 3
nlines = 2
l_num3, lines3 = al.get_most_relevant_lines_ion_state(ion_state = 3, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)
ion_energies = al.ion_energies

#compute the plasma temperature
temperature, temp_95, r2, y_s, x_s = sg.saha_boltzmann_temperature_v2([lines,lines2],ion_energies,electron_density= density, 
                                                                      ratio_of_maximum = 0.5, radius = 0.1, Plot = True, Plotlines = True, use_max_intensity = False)


# LTE conditions
Probing the Mcwrither criterium

In [ ]:
minimum_density = sg.LTE_MW_criterium(4.8,temperature)
print("Minimum density in cm^-3: " + str("%e" %minimum_density))
print("Computed density in cm^-3: " + str("%e" %(density*(10**-2)**3)))
print("MW criterion - " + str(density>(minimum_density*(10**2)**3)))

In [ ]:
al = element('Na')
sample_elements = [['Na',.5],['Cl',.5]]
sample = digital_twin(sample_elements)

print(sample)

for n_spectrometer in range(0,len(sg.wavelengths)):
    ll = sg.wavelengths[n_spectrometer][0]
    ul = sg.wavelengths[n_spectrometer][-1]

    #ion state 1
    nlines = 3
    l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, 
                                                        n_lines = nlines,lower_limit = ll, upper_limit = ul)

    #line to normalize the intensity for direct
    line_norm1=lines[1]
    print("# Spectrometer 1 normalized to " + str(line_norm1))

    sg.compare_to_digital_sample(sample, spectrometer=n_spectrometer, 
                                 electron_temperature = temperature,electron_density=density,
                                 d_lambda=0.02, use_wavelengths=True,line_normalize=line_norm1,Plotline = True)

In [ ]:
a=array([1,2])
b=copy(a)
b*=2
a

In [ ]:



ll = 200.
ul = 900.
Tp = 1.*T_ref
new_wavelengths = np.concatenate([wl for wl in sg.wavelengths])
#wl,spec,label, n_ion = al.synthetic_spectrum_NIST(wl = new_wavelengths, lower_limit = ll, upper_limit = ul, electron_temperature=Tp,max_ion_state=3,d_lambda=0.1)
wl,spec,label, n_ion, specs = sample.spectrum_NIST(wl = new_wavelengths, lower_limit = ll, upper_limit = ul, electron_temperature=Tp,max_ion_state=3,d_lambda=0.01, Map=True)
wl=wl[0]



In [ ]:
#define the upper and lower limits to find the most relevant nlines
#let us define to the
x_ss=[]
y_ss=[]
for n_spectrometer in range(0,6):
    ll = sg.wavelengths[n_spectrometer][0]
    ul = sg.wavelengths[n_spectrometer][-1]

    #ion state 1
    nlines = 5
    l_num, lines = al.get_most_relevant_lines_ion_state(ion_state = 1, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

    #ion state 2
    nlines = 2
    l_num2, lines2 = al.get_most_relevant_lines_ion_state(ion_state = 2, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)

    #ion state 3
    nlines = 2
    l_num3, lines3 = al.get_most_relevant_lines_ion_state(ion_state = 3, electron_temperature=Tp, n_lines = nlines,lower_limit = ll, upper_limit = ul)
    ion_energies = al.ion_energies

    #compute the plasma temperature
    temperature1, temp_95, r2, y_s, x_s = sg.saha_boltzmann_temperature_v2([lines,lines2],ion_energies,electron_density= density, 
                                                                          ratio_of_maximum = 0.5, radius = 0.1, Plot = False, Plotlines = False, use_max_intensity = False)

    x_ss.append(x_s)
    y_ss.append(y_s)